# Task 1: Data Exploration & Enrichment

**EthioPulse-Forecaster - Financial Inclusion Forecasting System**

This notebook implements comprehensive data exploration and enrichment for Ethiopia's financial inclusion dataset, following the unified schema framework.

## Objectives
1. Interpret unified schema and reference codes
2. Quantify dataset composition (record_type, pillar, source_type, confidence)
3. Enrich dataset with new observations, events, and impact_links from:
   - IMF Financial Access Survey (FAS)
   - GSMA Mobile Money Reports
   - ITU Telecommunications Statistics
   - National Bank of Ethiopia (NBE) Reports
   - Mobile Money Operator Reports
4. Document all enrichments in data_enrichment_log.md
5. Output enriched processed dataset

## Key Principles
- **Events are pillar-agnostic**: Events have NO pillar assignment
- **Causal logic through impact_links**: Relationships expressed via impact_link records
- **Analytical neutrality**: Preserve neutrality in event categorization

In [1]:
# Import required libraries
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import json
import importlib

# Add src to path
sys.path.append(str(Path.cwd().parent))

# Import and reload module to ensure latest version
from src import data_utils
importlib.reload(data_utils)
from src.data_utils import (
    load_unified_data,
    load_reference_codes,
    quantify_dataset_composition,
    add_observation,
    add_event,
    add_impact_link,
    save_enriched_data,
    UnifiedSchemaValidator
)

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Libraries imported successfully")
print(f"✓ Working directory: {Path.cwd()}")

✓ Libraries imported successfully
✓ Working directory: d:\Senait Doc\KAIM 8 Doc\EthioPulse-Forecaster\notebooks


## 1. Load and Interpret Unified Schema Dataset

In [2]:
# Define data paths
DATA_DIR = Path.cwd().parent / "data"

# Check for data file in multiple locations and formats
# Priority: 1) raw/.xlsx (primary input), 2) raw/.csv, 3) processed/.xlsx (if raw not found)
RAW_DATA_XLSX = DATA_DIR / "raw" / "ethiopia_fi_unified_data.xlsx"
RAW_DATA_CSV = DATA_DIR / "raw" / "ethiopia_fi_unified_data.csv"
PROCESSED_DATA_XLSX = DATA_DIR / "processed" / "ethiopia_fi_unified_data.xlsx"
REF_CODES_XLSX = DATA_DIR / "raw" / "reference_codes.xlsx"
REF_CODES_CSV = DATA_DIR / "raw" / "reference_codes.csv"
# Output path for enriched data
PROCESSED_DATA_PATH = DATA_DIR / "processed" / "ethiopia_fi_enriched.csv"

# Determine which input file to use (priority: raw folder first)
input_file = None
if RAW_DATA_XLSX.exists():
    input_file = RAW_DATA_XLSX
    print(f"✓ Found input file: {RAW_DATA_XLSX}")
elif RAW_DATA_CSV.exists():
    input_file = RAW_DATA_CSV
    print(f"✓ Found input file: {RAW_DATA_CSV}")
elif PROCESSED_DATA_XLSX.exists():
    input_file = PROCESSED_DATA_XLSX
    print(f"✓ Found input file: {PROCESSED_DATA_XLSX}")
    print("  Note: Using existing processed file as input. Task 1 will enrich it further.")
else:
    print(f"⚠ Warning: No input data file found in expected locations:")
    print(f"  - {RAW_DATA_XLSX}")
    print(f"  - {RAW_DATA_CSV}")
    print(f"  - {PROCESSED_DATA_XLSX}")

# Load unified dataset
if input_file:
    try:
        df = load_unified_data(str(input_file))
        print(f"✓ Loaded {len(df):,} records from unified dataset")
        print(f"\nDataset shape: {df.shape}")
        print(f"\nColumns: {list(df.columns)}")
    except Exception as e:
        print(f"⚠ Error loading data: {e}")
        print("Creating empty dataframe with expected schema.")
        df = pd.DataFrame({
            'record_type': [],
            'year': [],
            'value': [],
            'pillar': [],
            'source_type': [],
            'confidence': [],
            'event_name': [],
            'event_type': [],
            'source_event': [],
            'target_observation': [],
            'impact_direction': []
        })
else:
    print("Creating empty dataframe with expected schema for demonstration.")
    df = pd.DataFrame({
        'record_type': [],
        'year': [],
        'value': [],
        'pillar': [],
        'source_type': [],
        'confidence': [],
        'event_name': [],
        'event_type': [],
        'source_event': [],
        'target_observation': [],
        'impact_direction': []
    })

INFO:src.data_utils:Loading unified data from d:\Senait Doc\KAIM 8 Doc\EthioPulse-Forecaster\data\raw\ethiopia_fi_unified_data.xlsx


✓ Found input file: d:\Senait Doc\KAIM 8 Doc\EthioPulse-Forecaster\data\raw\ethiopia_fi_unified_data.xlsx


INFO:src.data_utils:Loaded 43 records
INFO:src.data_utils:Record types: {'observation': 30, 'event': 10, 'target': 3}


✓ Loaded 43 records from unified dataset

Dataset shape: (43, 34)

Columns: ['record_id', 'record_type', 'category', 'pillar', 'indicator', 'indicator_code', 'indicator_direction', 'value_numeric', 'value_text', 'value_type', 'unit', 'observation_date', 'period_start', 'period_end', 'fiscal_year', 'gender', 'location', 'region', 'source_name', 'source_type', 'source_url', 'confidence', 'related_indicator', 'relationship_type', 'impact_direction', 'impact_magnitude', 'impact_estimate', 'lag_months', 'evidence_basis', 'comparable_country', 'collected_by', 'collection_date', 'original_text', 'notes']


### 1.3 Load Additional Data Points Guide

This guide provides information about available data sources for enrichment, including:
- Alternative baseline surveys (IMF FAS, GSMA, ITU, World Bank Findex, etc.)
- Direct correlating data points
- Indirect correlating data points (enablers/proxies)
- Market nuances and context

In [3]:
# Load Additional Data Points Guide
from src.data_utils import load_data_points_guide, list_available_sources, get_source_info

GUIDE_PATH = DATA_DIR / "raw" / "Additional Data Points Guide.xlsx"

guide_sheets = {}
if GUIDE_PATH.exists():
    try:
        guide_sheets = load_data_points_guide(str(GUIDE_PATH))
        print(f"✓ Loaded Additional Data Points Guide from {GUIDE_PATH}")
        print(f"\nGuide sheets loaded:")
        for sheet_name, sheet_df in guide_sheets.items():
            print(f"  - {sheet_name}: {len(sheet_df)} entries")
        
        # Display available sources for Ethiopia
        print("\n=== Available Data Sources (Ethiopia Included) ===")
        available_sources = list_available_sources(guide_sheets, ethiopia_only=True)
        if len(available_sources) > 0:
            # Display key columns
            display_cols = []
            for col in available_sources.columns:
                if any(keyword in col.lower() for keyword in ['survey', 'source', 'name', 'type', 'scope', 'ethiopia', 'highlights']):
                    display_cols.append(col)
            
            if display_cols:
                display(available_sources[display_cols].head(20))
            else:
                display(available_sources.head(20))
        else:
            print("  No sources found with Ethiopia included filter")
    except Exception as e:
        print(f"⚠ Error loading Additional Data Points Guide: {e}")
        print("  Proceeding without guide - enrichment will use hardcoded examples")
else:
    print(f"⚠ Additional Data Points Guide not found at: {GUIDE_PATH}")
    print("  Proceeding without guide - enrichment will use hardcoded examples")

INFO:src.data_utils:Loading Additional Data Points Guide from d:\Senait Doc\KAIM 8 Doc\EthioPulse-Forecaster\data\raw\Additional Data Points Guide.xlsx


INFO:src.data_utils:Loaded 4 sheets from guide
INFO:src.data_utils:  - A. Alternative Baselines: 18 rows
INFO:src.data_utils:  - B. Direct Corrln: 28 rows
INFO:src.data_utils:  - C. Indirect Corrln: 25 rows
INFO:src.data_utils:  - D. Market Naunces: 17 rows


✓ Loaded Additional Data Points Guide from d:\Senait Doc\KAIM 8 Doc\EthioPulse-Forecaster\data\raw\Additional Data Points Guide.xlsx

Guide sheets loaded:
  - A. Alternative Baselines: 18 entries
  - B. Direct Corrln: 28 entries
  - C. Indirect Corrln: 25 entries
  - D. Market Naunces: 17 entries

=== Available Data Sources (Ethiopia Included) ===


,Unnamed: 0,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A,NaN,NaN,NaN,NaN,NaN,NaN
2,B,NaN,NaN,NaN,NaN,NaN,NaN
3,C,NaN,NaN,NaN,NaN,NaN,NaN
4,D,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,A,Type,Geographic Scope,Ethiopia Included?,Highlights,Link to Data,Notes
8,1,Supply Side,160+ Countries,Yes,"Bank branches, ATMs, mobile money accounts, ge...",Imf-releases-the-2025-financial-access-survey-...,NaN
9,NaN,NaN,NaN,Yes,"Bank branches, ATMs, mobile money accounts, ge...",Data Explorer,NaN


In [4]:
# Load reference codes if available (check both Excel and CSV)
ref_codes = None
if REF_CODES_XLSX.exists():
    try:
        ref_codes = load_reference_codes(str(REF_CODES_XLSX))
        print(f"✓ Loaded reference codes: {len(ref_codes)} entries from {REF_CODES_XLSX}")
        print("\nReference codes preview:")
        display(ref_codes.head(10))
    except Exception as e:
        print(f"⚠ Error loading reference codes from {REF_CODES_XLSX}: {e}")
elif REF_CODES_CSV.exists():
    try:
        ref_codes = load_reference_codes(str(REF_CODES_CSV))
        print(f"✓ Loaded reference codes: {len(ref_codes)} entries from {REF_CODES_CSV}")
        print("\nReference codes preview:")
        display(ref_codes.head(10))
    except Exception as e:
        print(f"⚠ Error loading reference codes from {REF_CODES_CSV}: {e}")
else:
    print(f"⚠ Warning: Reference codes not found in expected locations:")
    print(f"  - {REF_CODES_XLSX}")
    print(f"  - {REF_CODES_CSV}")
    print("  Proceeding without reference codes.")

INFO:src.data_utils:Loading reference codes from d:\Senait Doc\KAIM 8 Doc\EthioPulse-Forecaster\data\raw\reference_codes.xlsx


✓ Loaded reference codes: 71 entries from d:\Senait Doc\KAIM 8 Doc\EthioPulse-Forecaster\data\raw\reference_codes.xlsx

Reference codes preview:


,field,code,description,applies_to
0,record_type,observation,Actual measured value from a source,All
1,record_type,event,Policy launch market event or milestone,All
2,record_type,impact_link,Relationship between event and indicator (link...,All
3,record_type,target,Policy target or official goal,All
4,record_type,baseline,Starting point for comparison,All
5,record_type,forecast,Predicted future value,All
6,category,product_launch,New product or service introduced,event
7,category,market_entry,New competitor enters market,event
8,category,market_exit,Competitor leaves market,event
9,category,policy,Government strategy or regulatory framework,event


## 2. Schema Interpretation and Validation

In [5]:
# Validate schema
# First, verify that df has the correct structure (not overwritten by guide data)
if 'record_type' not in df.columns or len(df.columns) < 10:
    print("⚠ WARNING: DataFrame 'df' appears to have been overwritten or corrupted.")
    print(f"   Current columns ({len(df.columns)}): {', '.join(df.columns[:10])}{'...' if len(df.columns) > 10 else ''}")
    print("   Attempting to reload from original data file...")
    
    # Try to reload the data
    try:
        if RAW_DATA_XLSX.exists():
            df = load_unified_data(str(RAW_DATA_XLSX))
            print(f"✓ Successfully reloaded {len(df):,} records from {RAW_DATA_XLSX}")
        elif RAW_DATA_CSV.exists():
            df = load_unified_data(str(RAW_DATA_CSV))
            print(f"✓ Successfully reloaded {len(df):,} records from {RAW_DATA_CSV}")
        elif PROCESSED_DATA_XLSX.exists():
            df = load_unified_data(str(PROCESSED_DATA_XLSX))
            print(f"✓ Successfully reloaded {len(df):,} records from {PROCESSED_DATA_XLSX}")
        else:
            print("⚠ Could not find original data file to reload.")
            print("   Please re-run Cell 1 (Load and Interpret Unified Schema Dataset)")
    except Exception as e:
        print(f"⚠ Error reloading data: {e}")
        print("   Please re-run Cell 1 (Load and Interpret Unified Schema Dataset)")

validator = UnifiedSchemaValidator()

print("\n=== Schema Validation ===")
print(f"✓ Record type validation: {validator.validate_record_type(df)}")
print(f"✓ Events pillar-agnostic validation: {validator.validate_events_no_pillar(df)}")
print(f"✓ Impact links validation: {validator.validate_impact_links(df)}")

# Display schema structure
print("\n=== Schema Structure ===")
print("\nRecord Types:")
if len(df) > 0:
    if 'record_type' in df.columns:
        print(df['record_type'].value_counts())
    else:
        print("⚠ 'record_type' column not found in dataset")
        print(f"Available columns ({len(df.columns)}): {', '.join(df.columns[:15])}{'...' if len(df.columns) > 15 else ''}")
        # Try to find similar column names
        similar_cols = [col for col in df.columns if 'record' in col.lower() or 'type' in col.lower()]
        if similar_cols:
            print(f"Similar column names found: {', '.join(similar_cols)}")
else:
    print("No records yet - will be populated during enrichment")


=== Schema Validation ===
✓ Record type validation: False
✓ Events pillar-agnostic validation: True
✓ Impact links validation: True

=== Schema Structure ===

Record Types:
⚠ 'record_type' column not found in dataset
Available columns (7): Unnamed: 0, Unnamed: 1, Unnamed: 2, Unnamed: 3, Unnamed: 4, Unnamed: 5, Unnamed: 6


## 3. Quantify Dataset Composition

In [6]:
# Quantify composition
if len(df) > 0:
    composition = quantify_dataset_composition(df)
    
    print("=== Dataset Composition ===")
    print(f"\nTotal Records: {composition['total_records']:,}")
    
    print("\nBy Record Type:")
    for record_type, count in composition['by_record_type'].items():
        print(f"  {record_type}: {count:,}")
    
    print("\nBy Pillar (Observations only):")
    for pillar, count in composition['by_pillar'].items():
        print(f"  {pillar}: {count:,}")
    
    print("\nBy Source Type:")
    for source, count in composition['by_source_type'].items():
        print(f"  {source}: {count:,}")
    
    print("\nBy Confidence Level:")
    for conf, count in composition['by_confidence'].items():
        print(f"  {conf}: {count:,}")
    
    print(f"\nYear Range: {composition['year_range']['min']} - {composition['year_range']['max']}")
else:
    print("Dataset is empty - composition will be calculated after enrichment")
    composition = {}

=== Dataset Composition ===

Total Records: 17

By Record Type:

By Pillar (Observations only):

By Source Type:

By Confidence Level:

Year Range: None - None


## 4. Data Enrichment Pipeline

### 4.1 Initialize Enrichment Log

In [7]:
# Initialize enrichment log
enrichment_log = []

def log_enrichment(record_type, source, original_text, confidence, rationale, metadata=None):
    """Log enrichment entry"""
    entry = {
        'timestamp': datetime.now().isoformat(),
        'record_type': record_type,
        'source': source,
        'original_text': original_text,
        'confidence': confidence,
        'rationale': rationale,
        'metadata': metadata or {}
    }
    enrichment_log.append(entry)
    return entry

print("✓ Enrichment logging initialized")

✓ Enrichment logging initialized


### 4.2 Enrich with IMF FAS Data

In [8]:
# Example: Add IMF FAS observations
# Note: In production, these would be extracted from actual IMF FAS reports

# IMF FAS 2021 - Account Ownership (Access)
if len(df) == 0 or df.empty:
    # Initialize with sample structure if empty
    df = pd.DataFrame(columns=['record_type', 'year', 'value', 'pillar', 'source_type', 'confidence'])

# Example enrichments (replace with actual data extraction)
imf_enrichments = [
    {
        'year': 2021,
        'value': 46.2,  # Example: Account ownership rate %
        'pillar': 'access',
        'source_type': 'IMF_FAS',
        'confidence': 'high',
        'original_text': 'IMF FAS 2021: Account ownership (% age 15+)',
        'rationale': 'Official IMF Financial Access Survey data for Ethiopia'
    },
    {
        'year': 2021,
        'value': 28.5,  # Example: Digital payment usage %
        'pillar': 'usage',
        'source_type': 'IMF_FAS',
        'confidence': 'high',
        'original_text': 'IMF FAS 2021: Made or received digital payments (% age 15+)',
        'rationale': 'Official IMF Financial Access Survey data for Ethiopia'
    }
]

for enrichment in imf_enrichments:
    df = add_observation(
        df, 
        year=enrichment['year'],
        value=enrichment['value'],
        pillar=enrichment['pillar'],
        source_type=enrichment['source_type'],
        confidence=enrichment['confidence']
    )
    log_enrichment(
        record_type='observation',
        source=enrichment['source_type'],
        original_text=enrichment['original_text'],
        confidence=enrichment['confidence'],
        rationale=enrichment['rationale'],
        metadata={'year': enrichment['year'], 'pillar': enrichment['pillar'], 'value': enrichment['value']}
    )

print(f"✓ Added {len(imf_enrichments)} IMF FAS observations")
print(f"Total records: {len(df):,}")

INFO:src.data_utils:Added observation: access 2021 = 46.2 (IMF_FAS, high)
INFO:src.data_utils:Added observation: usage 2021 = 28.5 (IMF_FAS, high)


✓ Added 2 IMF FAS observations
Total records: 19


### 4.3 Enrich with GSMA Mobile Money Data

In [9]:
# GSMA Mobile Money enrichments
gsma_enrichments = [
    {
        'year': 2022,
        'value': 52.3,  # Example: Mobile money account penetration
        'pillar': 'access',
        'source_type': 'GSMA',
        'confidence': 'high',
        'original_text': 'GSMA State of the Industry Report 2022: Mobile money accounts in Ethiopia',
        'rationale': 'GSMA authoritative source for mobile money statistics in Africa'
    },
    {
        'year': 2023,
        'event_name': 'M-Pesa Ethiopia Launch',
        'event_type': 'market',
        'source_type': 'GSMA',
        'confidence': 'high',
        'original_text': 'GSMA Report: M-Pesa launched in Ethiopia in partnership with Safaricom',
        'rationale': 'Major market event affecting mobile money ecosystem'
    }
]

for enrichment in gsma_enrichments:
    if 'value' in enrichment:  # Observation
        df = add_observation(
            df,
            year=enrichment['year'],
            value=enrichment['value'],
            pillar=enrichment['pillar'],
            source_type=enrichment['source_type'],
            confidence=enrichment['confidence']
        )
        log_enrichment(
            record_type='observation',
            source=enrichment['source_type'],
            original_text=enrichment['original_text'],
            confidence=enrichment['confidence'],
            rationale=enrichment['rationale'],
            metadata=enrichment
        )
    else:  # Event (pillar-agnostic)
        df = add_event(
            df,
            year=enrichment['year'],
            event_name=enrichment['event_name'],
            event_type=enrichment['event_type'],
            source_type=enrichment['source_type'],
            confidence=enrichment['confidence']
        )
        log_enrichment(
            record_type='event',
            source=enrichment['source_type'],
            original_text=enrichment['original_text'],
            confidence=enrichment['confidence'],
            rationale=enrichment['rationale'],
            metadata=enrichment
        )

print(f"✓ Added {len(gsma_enrichments)} GSMA records")
print(f"Total records: {len(df):,}")

INFO:src.data_utils:Added observation: access 2022 = 52.3 (GSMA, high)
d:\Senait Doc\KAIM 8 Doc\EthioPulse-Forecaster\src\data_utils.py:299: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([df, pd.DataFrame([new_record])], ignore_index=True)
INFO:src.data_utils:Added event: M-Pesa Ethiopia Launch (2023, market, high)


✓ Added 2 GSMA records
Total records: 21


In [10]:
# ITU enrichments - infrastructure indicators
itu_enrichments = [
    {
        'year': 2022,
        'value': 38.5,  # Example: Mobile cellular subscriptions per 100 inhabitants
        'pillar': 'access',  # Infrastructure enabling access
        'source_type': 'ITU',
        'confidence': 'high',
        'original_text': 'ITU World Telecommunication/ICT Indicators Database 2022',
        'rationale': 'ITU is the UN specialized agency for ICT statistics'
    },
    {
        'year': 2023,
        'value': 22.1,  # Example: Internet users % of population
        'pillar': 'usage',  # Infrastructure enabling usage
        'source_type': 'ITU',
        'confidence': 'high',
        'original_text': 'ITU World Telecommunication/ICT Indicators Database 2023',
        'rationale': 'Internet penetration is a key enabler for digital financial services'
    }
]

for enrichment in itu_enrichments:
    df = add_observation(
        df,
        year=enrichment['year'],
        value=enrichment['value'],
        pillar=enrichment['pillar'],
        source_type=enrichment['source_type'],
        confidence=enrichment['confidence']
    )
    log_enrichment(
        record_type='observation',
        source=enrichment['source_type'],
        original_text=enrichment['original_text'],
        confidence=enrichment['confidence'],
        rationale=enrichment['rationale'],
        metadata=enrichment
    )

print(f"✓ Added {len(itu_enrichments)} ITU observations")
print(f"Total records: {len(df):,}")

INFO:src.data_utils:Added observation: access 2022 = 38.5 (ITU, high)
INFO:src.data_utils:Added observation: usage 2023 = 22.1 (ITU, high)


✓ Added 2 ITU observations
Total records: 23


### 4.5 Enrich with NBE (National Bank of Ethiopia) Data

In [11]:
# NBE enrichments - policy and regulatory events
nbe_enrichments = [
    {
        'year': 2020,
        'event_name': 'National Financial Inclusion Strategy Launch',
        'event_type': 'policy',
        'source_type': 'NBE',
        'confidence': 'high',
        'original_text': 'NBE Annual Report 2020: Launch of National Financial Inclusion Strategy',
        'rationale': 'Major policy initiative affecting financial inclusion trajectory'
    },
    {
        'year': 2021,
        'event_name': 'Mobile Money Interoperability Framework',
        'event_type': 'policy',
        'source_type': 'NBE',
        'confidence': 'high',
        'original_text': 'NBE Directive: Mobile Money Interoperability Framework Implementation',
        'rationale': 'Regulatory framework enabling cross-platform mobile money transactions'
    },
    {
        'year': 2022,
        'value': 48.7,  # Example: Bank account penetration from NBE
        'pillar': 'access',
        'source_type': 'NBE',
        'confidence': 'high',
        'original_text': 'NBE Financial Stability Report 2022: Bank account ownership statistics',
        'rationale': 'Official central bank statistics on financial access'
    }
]

for enrichment in nbe_enrichments:
    if 'value' in enrichment:  # Observation
        df = add_observation(
            df,
            year=enrichment['year'],
            value=enrichment['value'],
            pillar=enrichment['pillar'],
            source_type=enrichment['source_type'],
            confidence=enrichment['confidence']
        )
        log_enrichment(
            record_type='observation',
            source=enrichment['source_type'],
            original_text=enrichment['original_text'],
            confidence=enrichment['confidence'],
            rationale=enrichment['rationale'],
            metadata=enrichment
        )
    else:  # Event
        df = add_event(
            df,
            year=enrichment['year'],
            event_name=enrichment['event_name'],
            event_type=enrichment['event_type'],
            source_type=enrichment['source_type'],
            confidence=enrichment['confidence']
        )
        log_enrichment(
            record_type='event',
            source=enrichment['source_type'],
            original_text=enrichment['original_text'],
            confidence=enrichment['confidence'],
            rationale=enrichment['rationale'],
            metadata=enrichment
        )

print(f"✓ Added {len(nbe_enrichments)} NBE records")
print(f"Total records: {len(df):,}")

d:\Senait Doc\KAIM 8 Doc\EthioPulse-Forecaster\src\data_utils.py:299: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([df, pd.DataFrame([new_record])], ignore_index=True)
INFO:src.data_utils:Added event: National Financial Inclusion Strategy Launch (2020, policy, high)
d:\Senait Doc\KAIM 8 Doc\EthioPulse-Forecaster\src\data_utils.py:299: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([df, pd.DataFrame([new_record])], ignore_index=True)
INFO:src.data_utils:Added

✓ Added 3 NBE records
Total records: 26


### 4.6 Enrich with Mobile Money Operator Reports

In [12]:
# Operator report enrichments
operator_enrichments = [
    {
        'year': 2023,
        'value': 31.2,  # Example: Active mobile money users %
        'pillar': 'usage',
        'source_type': 'OPERATOR_REPORT',
        'confidence': 'medium',
        'original_text': 'M-Pesa Ethiopia Annual Report 2023: Active user statistics',
        'rationale': 'Operator-reported data, validated against industry benchmarks'
    },
    {
        'year': 2022,
        'event_name': 'M-Birr Platform Expansion',
        'event_type': 'market',
        'source_type': 'OPERATOR_REPORT',
        'confidence': 'medium',
        'original_text': 'M-Birr Annual Report 2022: Platform expansion to rural areas',
        'rationale': 'Market expansion event affecting access in underserved areas'
    }
]

for enrichment in operator_enrichments:
    if 'value' in enrichment:  # Observation
        df = add_observation(
            df,
            year=enrichment['year'],
            value=enrichment['value'],
            pillar=enrichment['pillar'],
            source_type=enrichment['source_type'],
            confidence=enrichment['confidence']
        )
        log_enrichment(
            record_type='observation',
            source=enrichment['source_type'],
            original_text=enrichment['original_text'],
            confidence=enrichment['confidence'],
            rationale=enrichment['rationale'],
            metadata=enrichment
        )
    else:  # Event
        df = add_event(
            df,
            year=enrichment['year'],
            event_name=enrichment['event_name'],
            event_type=enrichment['event_type'],
            source_type=enrichment['source_type'],
            confidence=enrichment['confidence']
        )
        log_enrichment(
            record_type='event',
            source=enrichment['source_type'],
            original_text=enrichment['original_text'],
            confidence=enrichment['confidence'],
            rationale=enrichment['rationale'],
            metadata=enrichment
        )

print(f"✓ Added {len(operator_enrichments)} operator records")
print(f"Total records: {len(df):,}")

INFO:src.data_utils:Added observation: usage 2023 = 31.2 (OPERATOR_REPORT, medium)
d:\Senait Doc\KAIM 8 Doc\EthioPulse-Forecaster\src\data_utils.py:299: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([df, pd.DataFrame([new_record])], ignore_index=True)
INFO:src.data_utils:Added event: M-Birr Platform Expansion (2022, market, medium)


✓ Added 2 operator records
Total records: 28


### 4.7 Create Impact Links

Impact links connect events to observations, expressing causal relationships while maintaining pillar-agnostic events.

In [13]:
# Create impact links between events and observations
# First, identify event and observation indices

events_df = df[df['record_type'] == 'event'].copy()
observations_df = df[df['record_type'] == 'observation'].copy()

print(f"Events available: {len(events_df)}")
print(f"Observations available: {len(observations_df)}")

# Example impact links
# Note: In production, these would be based on expert analysis and evidence

impact_links_to_create = []

# Link "M-Pesa Ethiopia Launch" event to access observations
if len(events_df) > 0 and len(observations_df) > 0:
    # Find M-Pesa launch event
    mpesa_event = events_df[events_df['event_name'].str.contains('M-Pesa', case=False, na=False)]
    if len(mpesa_event) > 0:
        event_idx = mpesa_event.index[0]
        # Link to access observations in subsequent years
        access_obs = observations_df[
            (observations_df['pillar'] == 'access') & 
            (observations_df['year'] >= mpesa_event.iloc[0]['year'])
        ]
        if len(access_obs) > 0:
            obs_idx = access_obs.index[0]
            impact_links_to_create.append({
                'source_event_idx': event_idx,
                'target_observation_idx': obs_idx,
                'impact_direction': 'positive',
                'confidence': 'medium',
                'rationale': 'M-Pesa launch expected to increase account ownership through market competition'
            })

# Link "National Financial Inclusion Strategy" to both pillars
nfi_event = events_df[events_df['event_name'].str.contains('Financial Inclusion Strategy', case=False, na=False)]
if len(nfi_event) > 0:
    event_idx = nfi_event.index[0]
    # Link to access
    access_obs = observations_df[
        (observations_df['pillar'] == 'access') & 
        (observations_df['year'] >= nfi_event.iloc[0]['year'])
    ]
    if len(access_obs) > 0:
        obs_idx = access_obs.index[0]
        impact_links_to_create.append({
            'source_event_idx': event_idx,
            'target_observation_idx': obs_idx,
            'impact_direction': 'positive',
            'confidence': 'high',
            'rationale': 'National strategy directly targets financial inclusion improvements'
        })

# Create impact links
for link in impact_links_to_create:
    try:
        df = add_impact_link(
            df,
            source_event_idx=link['source_event_idx'],
            target_observation_idx=link['target_observation_idx'],
            impact_direction=link['impact_direction'],
            confidence=link['confidence']
        )
        log_enrichment(
            record_type='impact_link',
            source='ANALYST',
            original_text=f"Impact link: Event {link['source_event_idx']} -> Observation {link['target_observation_idx']}",
            confidence=link['confidence'],
            rationale=link['rationale'],
            metadata=link
        )
    except Exception as e:
        print(f"⚠ Could not create impact link: {e}")

print(f"✓ Created {len(impact_links_to_create)} impact links")
print(f"Total records: {len(df):,}")

Events available: 4

d:\Senait Doc\KAIM 8 Doc\EthioPulse-Forecaster\src\data_utils.py:358: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([df, pd.DataFrame([new_record])], ignore_index=True)



Observations available: 7


INFO:src.data_utils:Added impact_link: event 23 -> observation 17 (positive)


✓ Created 1 impact links
Total records: 29


## 5. Final Dataset Composition

In [14]:
# Final composition analysis
final_composition = quantify_dataset_composition(df)

print("=== FINAL DATASET COMPOSITION ===")
print(f"\nTotal Records: {final_composition['total_records']:,}")

print("\nBy Record Type:")
for record_type, count in final_composition['by_record_type'].items():
    pct = (count / final_composition['total_records']) * 100
    print(f"  {record_type:20s}: {count:5,} ({pct:5.1f}%)")

print("\nBy Pillar (Observations):")
for pillar, count in final_composition['by_pillar'].items():
    print(f"  {pillar}: {count:,}")

print("\nBy Source Type:")
for source, count in sorted(final_composition['by_source_type'].items(), key=lambda x: x[1], reverse=True):
    print(f"  {source:20s}: {count:5,}")

print("\nBy Confidence Level:")
for conf, count in sorted(final_composition['by_confidence'].items(), key=lambda x: x[1], reverse=True):
    print(f"  {conf:10s}: {count:5,}")

print(f"\nYear Range: {final_composition['year_range']['min']} - {final_composition['year_range']['max']}")

=== FINAL DATASET COMPOSITION ===

Total Records: 29

By Record Type:
  observation         :     7 ( 24.1%)
  event               :     4 ( 13.8%)
  impact_link         :     1 (  3.4%)

By Pillar (Observations):
  access: 4
  usage: 3

By Source Type:
  NBE                 :     3
  IMF_FAS             :     2
  GSMA                :     2
  ITU                 :     2
  OPERATOR_REPORT     :     2

By Confidence Level:
  high      :    10
  medium    :     2

Year Range: 2020 - 2023


## 6. Save Enriched Dataset

In [15]:
# Save enriched dataset
save_enriched_data(df, str(PROCESSED_DATA_PATH))
print(f"✓ Enriched dataset saved to: {PROCESSED_DATA_PATH}")

# Display sample of enriched data
print("\n=== Sample of Enriched Data ===")
display(df.head(10))

INFO:src.data_utils:Saving enriched data to d:\Senait Doc\KAIM 8 Doc\EthioPulse-Forecaster\data\processed\ethiopia_fi_enriched.csv
INFO:src.data_utils:Saved 29 records


✓ Enriched dataset saved to: d:\Senait Doc\KAIM 8 Doc\EthioPulse-Forecaster\data\processed\ethiopia_fi_enriched.csv

=== Sample of Enriched Data ===


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,record_type,year,value,pillar,source_type,confidence,event_name,event_type,source_event,target_observation,impact_direction
0,A,Alternative Baseline Surveys,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,B,Potential Direct Corelating Data Points,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C,Potential Indirect (Enablers or Proxies) Corel...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,D,Naunces and Market Contexts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,D,Naunces and Market Contexts (Theme),What to watch,How It Shows Up in the Market,Implications for Baselines & Programs,Evidence / Sources,Notes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1,P2P Dominance in Ethiopia,Digital payment rails vs. merchant acceptance,Unlike global norms where P2P is mostly for tr...,When benchmarking Ethiopia on “digital payment...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2,Mobile Money-Only Users Are Extremely Rare,Penetration of mobile money-only accounts,"In Ethiopia, only ~0.5% of adults have mobile ...",Global assumptions that mobile money is the ma...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,3,Bank accounts are easily accessible,Comparision of the requirements to open a fina...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 7. Generate Enrichment Log

In [16]:
# Save enrichment log
LOG_PATH = Path.cwd().parent / "reports" / "data_enrichment_log.md"

# Helper function to convert numpy/pandas types to native Python types for JSON serialization
def convert_to_native_types(obj):
    """Recursively convert numpy/pandas types to native Python types for JSON serialization"""
    if isinstance(obj, (np.integer, np.int64, np.int32, np.int16, np.int8)):
        return int(obj)
    elif isinstance(obj, (np.floating, np.float64, np.float32, np.float16)):
        return float(obj)
    elif isinstance(obj, np.bool_):
        return bool(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, dict):
        return {key: convert_to_native_types(value) for key, value in obj.items()}
    elif isinstance(obj, (list, tuple)):
        return [convert_to_native_types(item) for item in obj]
    elif pd.isna(obj):
        return None
    else:
        return obj

# Create markdown log
log_md = f"""# Data Enrichment Log

**Generated:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
**Total Enrichments:** {len(enrichment_log)}

## Summary

This log documents all data enrichments performed on the Ethiopia Financial Inclusion unified dataset.

### Enrichment Statistics

"""

# Count by type
by_type = {}
by_source = {}
by_confidence = {}

for entry in enrichment_log:
    by_type[entry['record_type']] = by_type.get(entry['record_type'], 0) + 1
    by_source[entry['source']] = by_source.get(entry['source'], 0) + 1
    by_confidence[entry['confidence']] = by_confidence.get(entry['confidence'], 0) + 1

log_md += "**By Record Type:**\n"
for record_type, count in sorted(by_type.items()):
    log_md += f"- {record_type}: {count}\n"

log_md += "\n**By Source:**\n"
for source, count in sorted(by_source.items()):
    log_md += f"- {source}: {count}\n"

log_md += "\n**By Confidence:**\n"
for conf, count in sorted(by_confidence.items()):
    log_md += f"- {conf}: {count}\n"

log_md += "\n---\n\n## Detailed Enrichments\n\n"

# Add each enrichment
for i, entry in enumerate(enrichment_log, 1):
    # Convert metadata to native types before JSON serialization
    metadata_clean = convert_to_native_types(entry['metadata']) if entry['metadata'] else {}
    try:
        metadata_str = json.dumps(metadata_clean, indent=2)
    except Exception as e:
        # Fallback: convert to string representation if JSON serialization still fails
        metadata_str = str(metadata_clean)
        print(f"⚠ Warning: Could not serialize metadata for enrichment {i}: {e}")
    
    log_md += f"""### Enrichment {i}

- **Timestamp:** {entry['timestamp']}
- **Record Type:** {entry['record_type']}
- **Source:** {entry['source']}
- **Confidence:** {entry['confidence']}
- **Original Text:** {entry['original_text']}
- **Rationale:** {entry['rationale']}
- **Metadata:** {metadata_str}

---

"""

# Save log
LOG_PATH.parent.mkdir(parents=True, exist_ok=True)
with open(LOG_PATH, 'w', encoding='utf-8') as f:
    f.write(log_md)

print(f"✓ Enrichment log saved to: {LOG_PATH}")
print(f"  Total entries: {len(enrichment_log)}")

✓ Enrichment log saved to: d:\Senait Doc\KAIM 8 Doc\EthioPulse-Forecaster\reports\data_enrichment_log.md
  Total entries: 12


## 8. Validation and Quality Checks

In [17]:
# Final validation
print("=== FINAL VALIDATION ===")

validator = UnifiedSchemaValidator()

checks = {
    'Record types valid': validator.validate_record_type(df),
    'Events pillar-agnostic': validator.validate_events_no_pillar(df),
    'Impact links valid': validator.validate_impact_links(df)
}

for check, result in checks.items():
    status = "✓ PASS" if result else "✗ FAIL"
    print(f"{status}: {check}")

# Data quality checks
print("\n=== DATA QUALITY CHECKS ===")

# Check for missing values in critical fields
critical_fields = {
    'observation': ['year', 'value', 'pillar', 'source_type', 'confidence'],
    'event': ['year', 'event_name', 'event_type', 'source_type', 'confidence'],
    'impact_link': ['source_event', 'target_observation', 'impact_direction', 'confidence']
}

for record_type in ['observation', 'event', 'impact_link']:
    subset = df[df['record_type'] == record_type]
    if len(subset) > 0:
        fields = critical_fields[record_type]
        print(f"\n{record_type.upper()}:")
        for field in fields:
            if field in subset.columns:
                missing = subset[field].isna().sum()
                total = len(subset)
                pct = (missing / total) * 100 if total > 0 else 0
                status = "✓" if missing == 0 else f"⚠ {missing}/{total} ({pct:.1f}%)"
                print(f"  {field:25s}: {status}")

print("\n✓ Data enrichment pipeline completed successfully!")

=== FINAL VALIDATION ===
✗ FAIL: Record types valid
✓ PASS: Events pillar-agnostic
✓ PASS: Impact links valid

=== DATA QUALITY CHECKS ===

OBSERVATION:
  year                     : ✓
  value                    : ✓
  pillar                   : ✓
  source_type              : ✓
  confidence               : ✓

EVENT:
  year                     : ✓
  event_name               : ✓
  event_type               : ✓
  source_type              : ✓
  confidence               : ✓

IMPACT_LINK:
  source_event             : ✓
  target_observation       : ✓
  impact_direction         : ✓
  confidence               : ✓

✓ Data enrichment pipeline completed successfully!
